In [41]:
!pip install py3-tts SpeechRecognition langchain langchain-community pyaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.6/738.6 kB 264.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 662.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 370.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 527.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 508.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 749.0 kB/s eta 0:00:00--:--
  Attempting uninstall: pyobjc-core
    Found existing installation: pyobjc-core 10.2
    Uninstalling pyobjc-core-10.2:
      Successfully uninstalled pyobjc-core-10.2
  Attempting uninstall: pyobjc-framework-libdispatch
    Found existing installation: pyobjc-framework-libdispatch 10.2
    Uninstalling pyobjc-framework-libdispatch-10.2:
      Successfully uninstalled pyobjc-framework-libdispatch-10.2
  Attempting uninstall: pyobjc-framework-Cocoa
    Found exis

In [17]:
import speech_recognition as sr
import pyttsx3

from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory

In [15]:
# STT
print("Say something!")
def human():
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        audio = recognizer.listen(source)
        
        try:
            text = recognizer.recognize_google(audio)
            print(f"You said: {text}")
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service: {e}")
            
    return text

# human_output = human()
# human_output

Say something!


In [37]:
# AI Prompt Template
llm = Ollama(model="llama2")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chatbot assistant. Keep your responses concise and relevant. Do not start responses with either 'Assisstant:' or 'Bot:' or 'AI:'."),
    MessagesPlaceholder(variable_name="chat_history")
])
output_parser = StrOutputParser()
chat_history_memory = ChatMessageHistory()

# Chain
chain = prompt | llm | output_parser

In [38]:
def update_chat_history(text, human=True):
    if human:
        chat_history_memory.add_user_message(text)
    else:
        chat_history_memory.add_ai_message(text)

In [39]:
# TTS
def ai(chain, chat_history_memory):
    response = chain.invoke({"chat_history": chat_history_memory.messages})
    print(f"Assistant said: {response}")
    
    engine = pyttsx3.init()
    engine.say(response)
    engine.runAndWait()
    engine.stop()
    
    return response

# ai_output = ai(chain, human_output)
# ai_output

In [43]:
# Flow
human_output, ai_output = "", ""

print("Transcript")
while True:
    human_output = human()
    if human_output == "thanks bye":
        break
    update_chat_history(human_output)
    ai_output = ai(chain, chat_history_memory)
    update_chat_history(ai_output, human=False)

Transcript
You said: hello how are you
Assistant said: Hello! I'm doing well, thanks for asking! How about you?
You said: I'm also ok
Assistant said: Great to hear that you're both doing well! Is there anything else you'd like to chat about or ask me? I'm here to help with any questions you may have.
You said: yeah there's a project I'm trying to work on and I'd like your thoughts on it
Assistant said: AI: Of course! I'd be happy to help you with your project. Can you tell me more about it? What is it about and what are you hoping to achieve?
You said: yeah I'm trying to build an image classifier that can detect the presence of cancer cells in lung cells so I'm not sure that's the right part or I don't know yeah what are your thoughts on this
Assistant said: AI: Great, building an image classifier for cancer cell detection is a fascinating project! Can you tell me more about the dataset you're working with and any specific requirements you have for the classifier? This will help me pro

UnboundLocalError: local variable 'text' referenced before assignment

In [44]:
chat_history_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='hello how are you'), AIMessage(content="Hello! I'm doing well, thanks for asking! How about you?"), HumanMessage(content="I'm also ok"), AIMessage(content="Great to hear that you're both doing well! Is there anything else you'd like to chat about or ask me? I'm here to help with any questions you may have."), HumanMessage(content="yeah there's a project I'm trying to work on and I'd like your thoughts on it"), AIMessage(content="AI: Of course! I'd be happy to help you with your project. Can you tell me more about it? What is it about and what are you hoping to achieve?"), HumanMessage(content="yeah I'm trying to build an image classifier that can detect the presence of cancer cells in lung cells so I'm not sure that's the right part or I don't know yeah what are your thoughts on this"), AIMessage(content="AI: Great, building an image classifier for cancer cell detection is a fascinating project! Can you tell me more about the d

In [33]:
chat_history_memory.clear()

### TODO
1. Create a history aware chain. Append both AI and Human responses into the chat history.
2. Add AI response TTS.
3. Create a flow of events for STT to TTS.
4. Add Retriever (Future) https://python.langchain.com/v0.1/docs/use_cases/chatbots/quickstart/
5. Save chat history.